# Updating Conda Packages to the Latest Version

This notebook makes it easy to update package versions for my favourite packages. The JupyterLab Docker image I use is maintained here: https://github.com/hughcameron/jupyterlab

To update packages I'll run `conda update --all -y` within the shell of the running container. This command is also used in the Dockerfile.

By running the cells below and commiting the contents of `environment.yml` back to GitHub a 

In [1]:
!conda env export --name base > full_environment.yml

In [2]:
!wget https://raw.githubusercontent.com/hughcameron/jupyterlab/master/environment.yml

--2020-03-11 10:13:24--  https://raw.githubusercontent.com/hughcameron/jupyterlab/master/environment.yml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 981 [text/plain]
Saving to: ‘environment.yml’

environment.yml     100%[===================>]     981  --.-KB/s    in 0s      

2020-03-11 10:13:24 (125 MB/s) - ‘environment.yml’ saved [981/981]



In [3]:
import yaml

In [4]:
with open('full_environment.yml', 'r') as f:
    full_env = yaml.load(f.read())

with open('environment.yml', 'r') as f:
    git_env = yaml.load(f.read())

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [5]:
def list_deps(env): return {pack.split('=')[0]:pack for pack in env['dependencies'] if type(pack) is str}

In [6]:
full_deps = list_deps(full_env)

In [7]:
git_deps = list_deps(git_env)

In [8]:
package_updates = {k:v for k,v in full_deps.items() if k in git_deps.keys()}

In [9]:
dependancy_updates = []
for pack in git_env['dependencies']:
    if type(pack) is not str:
        dependancy_updates.append(pack)
    else:
        pack_name = pack.split('=')[0]
        if pack_name in package_updates.keys():
            dependancy_updates.append(package_updates[pack_name])
        else:
            dependancy_updates.append(pack_name)

In [10]:
git_env['dependencies'] = dependancy_updates

In [11]:
with open('environment.yml', 'w') as f:
    git_env = f.write(yaml.dump(git_env))